In [6]:
!pip install torch
!pip install torchaudio
!pip install transformers
!pip install PySoundFile


     -------------------------------------- 671.8/671.8 kB 5.3 MB/s eta 0:00:00
     -------------------------------------- 178.8/178.8 kB 3.6 MB/s eta 0:00:00
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)


In [7]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
model = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def speech_file_to_array_fn(batch: dict) -> dict:
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0]
    batch["sampling_rate"] = sampling_rate
    return batch

def resample(batch: dict) -> dict:
    resampler=torchaudio.transforms.Resample(batch['sampling_rate'], 16_000)
    batch["speech"] = resampler(batch["speech"]).numpy()
    batch["sampling_rate"] = 16_000
    return batch

def prepare_dataset(batch: dict) -> dict:
    # check that all files have the correct sampling rate
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"]).input_values
    return batch

def Wav2Vec2_larynx(file: str, tokenized: bool = False) -> str:
    b = {}
    b['path'] = file
    a = prepare_dataset(resample(speech_file_to_array_fn(b)))
    input_dict = processor(a["input_values"][0], return_tensors="pt", padding=True)
    logits = model(input_dict.input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]

    if tokenized:
        txt = processor.decode(pred_ids)
    else:
        txt = processor.decode(pred_ids).replace(' ','')
    return txt

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\theopPleum\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
# For use here!!!!
import torchaudio
from wav2vec2 import Wav2Vec2_larynx


variableX1 = 'out-1.wav' 
'''
In the API this file it will upload to server 
Then, passed to Wav2Vec2_larynx function 
return txt to part FRONT-END
'''

variableX2 = Wav2Vec2_larynx(variableX1, True)

print(variableX2)

111


RuntimeError: No audio I/O backend is available.